In [6]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain

import warnings
warnings.filterwarnings('ignore')
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['OPENAI_API_KEY']

# import azure_authentication_client
from langchain.chat_models import ChatOpenAI
import langchain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import StructuredTool,tool

In [ ]:
!pip install duckduckgo-search

In [13]:
# This is our internal authentication method. You will need to insert your own API key to interact with GPT or instantiate a local model.
# Please refer to LangChain docs on how to authenticate and instantiate a model.

# azure_authentication_client.authenticate_openai()
llm = ChatOpenAI(temperature=0)
llm.predict('hi, are you gpt 3.5 turbo? Please be respectful while you answer.')

'Hello! I am an AI language model developed by OpenAI, but I am not specifically GPT-3.5 Turbo. However, I am designed to provide helpful and respectful responses to your questions. How can I assist you today?'

In [16]:
tools = load_tools(["ddg-search", "llm-math"], llm=llm)

# Implementing our own custom tool

In [17]:
@tool(return_direct=True)
def cvss_score_to_severity(input:str) -> str:
    """
    A tool that can transform a cvss numerical score to a severity category.
    The input should be a real number and the output will be a string
    For Example 0.2->Low 8.3->High
    """
    input = float(input)
    if input<0.1:
        return "None"
    elif 0.1<=input<=3.9:
        return "Low"
    elif 3.9<input<=6.9:
        return "Medium"
    elif 6.9<input<=8.9:
        return "High"
    elif 8.9<input:
        return "Critical"

cvss_score_to_severity

StructuredTool(name='cvss_score_to_severity', description='cvss_score_to_severity(input: str) -> str - A tool that can transform a cvss numerical score to a severity category. \n    The input should be a real number and the output will be a string\n    For Example 0.2->Low 8.3->High', args_schema=<class 'pydantic.main.cvss_score_to_severitySchemaSchema'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<function cvss_score_to_severity at 0x7d06987127a0>, coroutine=None)

In [18]:
tools.append(cvss_score_to_severity)
agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [20]:
user_query = "What is the CVSS score of CVE-2021-44228? What is the severity of this score?"

In [21]:
langchain.debug=False
agent.run(user_query)



> Entering new AgentExecutor chain...
Thought: I need to find the CVSS score and severity of CVE-2021-44228.
Action:
```
{
  "action": "duckduckgo_search",
  "action_input": "CVSS score of CVE-2021-44228"
}
```

Observation: CVE-2021-44228 Detail. CVE-2021-44228. Detail. Modified. This vulnerability has been modified since it was last analyzed by the NVD. It is awaiting reanalysis which may result in further changes to the information provided. A critical remote code execution vulnerability in Apache Log4j (a logging tool used in many Java applications) was disclosed on December 9, 2021. This vulnerability is described in CVE-2021-44228. A critical remote code execution vulnerability in Apache Log4j (a logging tool used in many Java applications) was disclosed on December 9, 2021. Kelyan Yesil Penetration Tester Updated July 12, 2023 Unless you've been on a sabbatical for the past year, you probably know how a critical vulnerability known as Log4shell took over the world. The popular

'Critical'

In [ ]:
langchain.debug=True
agent.run(user_query)